# CAPÍTULO 4.

## FUNCIONES QUE SE PROGRAMARÁN EN EL CODIGO:

<img src="Img/MRG.png">

<img src="Img/VFGC.png">

<img src="Img/VDGC.png">

In [7]:
#############
# Librerias #
#############

import math
import numpy as np
from ArchivoMiembroArmadura import MiembroArmadura


#########
# Clase #
#########

class AnalisisMatricial():
    '''  
    Descripción:
    Esta clase sirve para resolver cualquier armadura en 2D, tanto isostatica como hiperestatica. Tambien nos permite acceder 
    a las propiedades de cuaquiera de sus elementos y a las reacciones, desplazamientos, fuerzas internas, etc.
    
    Funciones:
    --> VectorCoordenadasGlobales(self)
    --> Matriz_PI(self)
    --> Armadura(self)
    --> MatrizRigidezGlobal(self)
    --> MatrziRigidezGlobalParciales(self)
    --> VecFrzasGlobalesConocidas(self):
    --> VecDespGlobalesConocidas(self)
    --> y más ...
    
    Programó:
    Ing. Josue Emmanuel Cruz Barragan
    '''
    def __init__(self, tbl_Elem, tbl_Nods, tbl_Frza, tbl_Desp):
        
        ####################
        # Datos de entrada #
        ####################
        
        self.tE = tbl_Elem # Tabla de elementos.
        self.tN = tbl_Nods # Tabla de nodos.
        self.tF = tbl_Frza # Tabla de fuerzas.
        self.tD = tbl_Desp # Tabla de desplazamientos.
        
        ###############
        # Propiedades #
        ###############
        
        self.nE = len(self.tE) # Número de elementos en la estructura.
        self.nN = len(self.tN) # Número de nodos en la estructura.
        self.nGl = self.nN*2 # Número de grados de libertad en la estructura.
        # [Número de reacciones, Número de fuerzas conocidas, Diccionario de nodos]
        [self.nR, self.nFc, self.N] = self.VectorCoordenadasGlobales()
        self.PI = np.array(self.Matriz_PI()) # Matriz PI.
        self.Armad = self.Armadura() # Lista de las propiedades de los elementos.
        self.Kg = self.MatrizRigidezGlobal() # Matriz de rigidez global del elemento.
        [self.k11, self.k12, self.k21, self.k22] = self.MatrziRigidezGlobalParciales() # Sub-matrices de rigidez global.
        self.Fc = self.VecFrzasGlobalesConocidas() # Vector de fuerzas globales conocidas.
        self.Dc = self.VecDespGlobalesConocidas() # Vector de desplazamientos globales conocidos.
        
        
        #########################
        # Funciones de la clase #
        #########################
        
    def __str__(self):
        print("Numero de Elementos: ", self.nE)
        print("Numero de Nodos: ", self.nN)
        print("Numero de Grados de Libertad: ", self.nGl)
        print("Numero de Reacciones: ", self.nR)
        print("Numero de Fuerzas Conocidas: ", self.nFc)
        print("Diccionario de Nodos: ", self.N)
        print("\n Matriz PI: \n", self.PI)
        print("\n Elemento 3: \n", self.Armad[2])
        print("\n Matriz de Rigidez Global de la Armadura: \n", self.Kg)
        print("")
        print("\n Matriz de Rigidez k11: \n",self.k11)
        print("\n Matriz de Rigidez k12: \n",self.k12)
        print("\n Matriz de Rigidez k21: \n",self.k21)
        print("\n Matriz de Rigidez k22: \n",self.k22)
        print("")
        print("\n Vector de Fuerzas Conocidas: \n",self.Fc)
        print("")
        print("\n Vector de Desplazamientos Conocidos: \n",self.Dc)
        return ""
    
    def VectorCoordenadasGlobales(self):
        """Calcula el número de reacciones, número de fuerzas conocidas y un diccionario de nodos."""
        numReacciones = 0
        DiccionarioNodos = {}
        c = 1
        gl = self.nGl
        
        for i in range(self.nN):
            tipo = self.tN[i][3]
            
            NODO_key = self.tN[i][0]
            cx = self.tN[i][1]
            cy = self.tN[i][2]
            
            
            if tipo == "Libre":
                numReacciones += 0
                DiccionarioNodos.setdefault(NODO_key, [cx, cy, c, c+1])
                c+=2
                
            elif tipo == "Fijo":
                numReacciones += 2
                DiccionarioNodos.setdefault(NODO_key, [cx, cy, gl-1, gl])
                gl-=2
            
            elif tipo == "DX":
                numReacciones += 1
                DiccionarioNodos.setdefault(NODO_key, [cx, cy])
            
            elif tipo == "DY":
                numReacciones += 1
                DiccionarioNodos.setdefault(NODO_key, [cx, cy])
                
        numFuerzasConocidas = self.nGl - numReacciones  
        return numReacciones, numFuerzasConocidas, DiccionarioNodos
    
    def Matriz_PI(self):
        """Calcula la matriz de permutación PI."""
        PI = []
        for i in range(self.nE):
            ni = self.tE[i][3]
            nf = self.tE[i][4]
            PI.append([self.N[ni][2],self.N[ni][3],self.N[nf][2],self.N[nf][3]])
        return PI
    
    def Armadura(self):
        """Calcula la lista de propiedades de cada elemento de la armadura."""
        Elem = []
        for i in range(self.nE):
            el = self.tE[i][0]
            a = self.tE[i][1]
            me = self.tE[i][2]
            NI = self.tE[i][3]
            NF = self.tE[i][4]
            xi = self.N[NI][0]
            yi = self.N[NI][1]
            xf = self.N[NF][0]
            yf = self.N[NF][1]
            clix = self.N[NI][2]
            cliy = self.N[NI][3]
            clfx = self.N[NF][2]
            clfy = self.N[NF][3]
            Elem.append(MiembroArmadura(el, a, me, xi, yi, xf, yf, [clix, cliy, clfx, clfy]))
        return Elem
    
    def MatrizRigidezGlobal(self):
        """Calcula la matriz de rigidez global de la armadura."""
        K = np.zeros((self.nGl, self.nGl))
        for e in range(self.nE):
            ke_global = self.Armad[e].k_glob
            for i in range(4):
                for j in range(4):
                    a = self.PI[e,i]-1
                    b = self.PI[e,j]-1
                    K[a,b] = ke_global[i,j] + K[a,b]
        return K
    
    def MatrziRigidezGlobalParciales(self):
        """Calcula las sub-matrices de rigidez global de la armadura."""
        k11 = self.Kg[0:self.nFc, 0:self.nFc]
        k12 = self.Kg[0:self.nFc, self.nFc:self.nGl]
        k21 = self.Kg[self.nFc:self.nGl, 0:self.nFc]
        k22 = self.Kg[self.nFc:self.nGl, self.nFc:self.nGl]
        return k11, k12, k21, k22
    

    def VecFrzasGlobalesConocidas(self):
        """Calcula el vector de fuerzas globales conocidas de la armadura."""
        Fc = np.zeros((self.nFc,1))
        for i in range(len(self.tF)):
            direccion = self.tF[i][2] 
            nodo = self.tF[i][1]
            if direccion == 'DX':
                j = self.N[nodo][2]-1
            elif direccion == 'DY':
                j = self.N[nodo][3]-1
            Fc[j] = self.tF[i][0]
        return Fc
    
    
    def VecDespGlobalesConocidas(self):
        """Calcula el vector de desplazamientos globales conocidos de la armadura."""
        Dc = np.zeros((self.nGl - self.nFc,1))
        for i in range(len(self.tD)):
            direccion = self.tD[i][2] 
            nodo = self.tD[i][1]
            if direccion == 'DX':
                j = self.N[nodo][2]- 1 - self.nFc
            elif direccion == 'DY':
                j = self.N[nodo][3]- 1 - self.nFc
            Dc[j] = self.tD[i][0]
        return Dc


## EJEMPLO.

<img src="Img/Ejemplo.png">

In [8]:
A = 2 # Área en, m2
E = 1800 # Modulo de Elasticidad en, kgf/m2

# Tabla de Elementos:
tbl_Elem = [
    ['E1', A, E, 'N5', 'N1'],
    ['E2', A, E, 'N2', 'N1'],
    ['E3', A, E, 'N3', 'N2'],
    ['E4', A, E, 'N2', 'N5'],
    ['E5', A, E, 'N3', 'N5'],
    ['E6', A, E, 'N4', 'N5'],
    ['E7', A, E, 'N4', 'N3']]
     
# Tabla de Nodos:
tbl_Nods = [
    ['N1', 14, 7, "Libre"],
    ['N2', 7, 7, "Libre"],
    ['N3', 0, 7, "Fijo"],
    ['N4', 0, 0, "Fijo"],
    ['N5', 7, 0, "Libre"]]

# Tabla de Fuerzas:
tbl_Frza = [
    [ -3, 'N2', 'DY'],
    [ -2, 'N1', 'DY']]

# Tabla de Desplazamientos:
tbl_Desp = [
    [ 0, 'N3', 'DX'],
    [ 0, 'N3', 'DY'],
    [ 0, 'N4', 'DX'],
    [ 0, 'N4', 'DY']]

AE = AnalisisMatricial(tbl_Elem, tbl_Nods, tbl_Frza, tbl_Desp)
print(AE)

Numero de Elementos:  7
Numero de Nodos:  5
Numero de Grados de Libertad:  10
Numero de Reacciones:  4
Numero de Fuerzas Conocidas:  6
Diccionario de Nodos:  {'N1': [14, 7, 1, 2], 'N2': [7, 7, 3, 4], 'N3': [0, 7, 9, 10], 'N4': [0, 0, 7, 8], 'N5': [7, 0, 5, 6]}

 Matriz PI: 
 [[ 5  6  1  2]
 [ 3  4  1  2]
 [ 9 10  3  4]
 [ 3  4  5  6]
 [ 9 10  5  6]
 [ 7  8  5  6]
 [ 7  8  9 10]]

 Elemento 3: 
 Area:  2
Mod. Elasticidad:  1800
Coordenadas Iniciales: (0 , 7)
Coordenadas Finales: (7 , 7)
Vector de Cooredenadas:  [9, 10, 3, 4]
Longitud:  7.0
Rigidez Local: [[ 514.28571429 -514.28571429]
 [-514.28571429  514.28571429]]
Lx =  1.0   Ly =  0.0
Matriz de Transformacion:  [[1. 0. 0. 0.]
 [0. 0. 1. 0.]]

Rigidez global del elemento: [[ 514.28571429    0.         -514.28571429    0.        ]
 [   0.            0.            0.            0.        ]
 [-514.28571429    0.          514.28571429    0.        ]
 [   0.            0.            0.            0.        ]]


 Matriz de Rigidez Global de